In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [65]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [66]:
cat = Table.read('./Catalog/NSA/z002_004/UDG_NSA_MW_z002_004_JG.fits')

In [67]:
len(cat)

254

In [68]:
category = "UDG_new"

### Generate RGB images

In [69]:
from kuaizi.display import display_HSC_cutout_rgb

In [70]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_FIG_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/{category}/'

In [71]:
problem_ids = []
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            #print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")
            problem_ids.append(obj['viz-id'])
            
if not os.path.isdir(FIGURE_DIR):
    os.mkdir(FIGURE_DIR)

In [72]:
print('# of no cutouts:', np.unique(problem_ids).shape)

# of no cutouts: (0,)


In [45]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'griz'])
        fig, ax = plt.subplots(figsize=(5, 5))
        size = int(obj['rhalf_ellip'] * 5) if obj['rhalf_ellip'] * 5 > 100 else 100
        ax = display_HSC_cutout_rgb(images, ax=ax, half_width=size)
        plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 297
Progress: 10 / 297
Progress: 20 / 297
Progress: 30 / 297
Progress: 40 / 297
Progress: 50 / 297
Progress: 60 / 297
Progress: 70 / 297
Progress: 80 / 297
Progress: 90 / 297
Progress: 100 / 297
Progress: 110 / 297
Progress: 120 / 297
Progress: 130 / 297
Progress: 140 / 297
Progress: 150 / 297
Progress: 160 / 297
Progress: 170 / 297
Progress: 180 / 297
Progress: 190 / 297
Progress: 200 / 297
Progress: 210 / 297
Progress: 220 / 297
Progress: 230 / 297
Progress: 240 / 297
Progress: 250 / 297
Progress: 260 / 297
Progress: 270 / 297
Progress: 280 / 297
Progress: 290 / 297


In [73]:
# Copy figures in FIGURE_DIR to WEBPAGE_FIG_DIR
if not os.path.isdir(WEBPAGE_FIG_DIR):
    os.mkdir(WEBPAGE_FIG_DIR)
else:
    import glob
    fileList = glob.glob(WEBPAGE_FIG_DIR + '*.png', recursive=True)
    [os.remove(file) for file in fileList];
            
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try: 
        copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_FIG_DIR + f'nsa_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

### Generate webpage for cutouts. We use `diezi/webpage/gen_web_cutout.py`

In [74]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [75]:
#FOR NSA SAMPLE
WEBPAGE_FIG_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/{category}/'
WEBPAGE_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/{category}/'
SCARLET_DIR = None #'/tigress/jiaxuanl/public_html/NSA/scarlet_result/'
sample_name = 'NSA'

In [76]:
webpage_cutout(WEBPAGE_FIG_DIR, WEBPAGE_DIR, sample_name, sample_title=f'{category}', SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 254 galaxies to be displayed
Total pages: 11


In [77]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/UDG_new/


### The search box on webpage still doesn't work. Need to work on the javascript file